전체 디스크립터 중 모든 값이 0인 디스크립터의 비율 확인

In [1]:
import os
import sys
import argparse
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import open3d.visualization as vis
import torch
from scipy.spatial.distance import cdist
from scipy.spatial import cKDTree

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [89]:
class mdgat_kitti_data_set():
    def __init__(self, opt):
        self.kitti_path = opt.kitti_path
        self.mdgat_path = opt.mdgat_path
        self.seq_list = opt.seq_list
        self.memory_is_enough = opt.memory_is_enough
        self.mutual_check = opt.mutual_check
        self.transform_opt = opt.transform_opt
        self.fpfh_normal_radiuse = opt.fpfh_normal_radiuse
        self.fpfh_descriptors_radiuse = opt.fpfh_descriptors_radiuse

        self.dataset = []
        self.calib = {}
        self.pose = {}
        self.kp = {}
        self.desc = {}
        self.scores = {}
        self.scan = {}
        self.random_sample_num = 16384 #16384
        self.threshold = 0.5

        self._load_kitti_gt_txt()
        self._load_preprocessed_data()

        print('Data loaded')
        # for seq in self.seq_list:
            # print('    Sequence %02d has %d pose/ %d kp'%(seq, len(self.kp[seq]), self.kp[seq][0].shape[0]))

    def _load_kitti_gt_txt(self):
        '''
        :param txt_root:
        :param seq
        :return: [{anc_idx: *, pos_idx: *, seq: *}]                
        '''
        for seq in self.seq_list:
            with open(os.path.join(self.mdgat_path, 'preprocess-random-full','%02d'%seq, 'groundtruths.txt'), 'r') as f:
                lines_list = f.readlines()
                for i, line_str in enumerate(lines_list):
                    if i == 0:
                        # skip the header line
                        continue
                    line_splitted = line_str.split()
                    anc_idx = int(line_splitted[0])
                    pos_idx = int(line_splitted[1])

                    data = {'seq': seq, 'anc_idx': anc_idx, 'pos_idx': pos_idx}
                    self.dataset.append(data)
    
    def _load_preprocessed_data(self):
        for seq in self.seq_list:
            sequence = '%02d'%seq
            calibpath = os.path.join(self.mdgat_path, 'calib/sequences', sequence, 'calib.txt')
            posepath = os.path.join(self.mdgat_path, 'poses', '%02d.txt'%seq)
            with open(calibpath, 'r') as f:
                for line in f.readlines():
                    _, value = line.split(':', 1)
                    try:
                        calib = np.array([float(x) for x in value.split()])
                    except ValueError:
                        pass
                    calib = np.reshape(calib, (3, 4))    
                    self.calib[sequence] = np.vstack([calib, [0, 0, 0, 1]])
            
            poses = []
            with open(posepath, 'r') as f:
                for line in f.readlines():
                    T_w_cam0 = np.fromstring(line, dtype=float, sep=' ')
                    T_w_cam0 = T_w_cam0.reshape(3, 4)
                    T_w_cam0 = np.vstack((T_w_cam0, [0, 0, 0, 1]))
                    poses.append(T_w_cam0)
                self.pose[sequence] = poses

            '''If memory is enough, load all the data'''
            if self.memory_is_enough:
                kps = []
                scores = []
                desc = []
                folder = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence)
                folder = os.listdir(folder)   
                folder.sort(key=lambda x:int(x[:-4]))
                for idx in range(len(folder)):
                    file = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence, folder[idx])
                    if os.path.isfile(file):
                        pc = np.reshape(np.fromfile(file, dtype=np.float32), (-1, 37))
                        ones = np.ones((pc.shape[0], 1))
                        kps.append(np.concatenate((pc[:,:3], ones), axis=1))
                        scores.append(pc[:,3])
                        desc.append(pc[:,4:])
                    else:
                        kps.append([0])

                self.kp[sequence] = kps
                self.desc[sequence] = desc
                self.scores[sequence] = scores

    def _get_kitti_data(self, sequence, index_in_seq):
        pc_file = os.path.join('/media/vision/Seagate/DataSets/kitti/dataset/sequences', sequence, "velodyne" ,'%06d.bin' % index_in_seq)
        pc = np.fromfile(pc_file, dtype=np.float32)
        pc = pc.reshape((-1, 4))

        ones = np.ones((pc.shape[0], 1))
        pc = np.concatenate((pc[:,:3], ones), axis=1)
        pc = torch.tensor(pc, dtype=torch.double)
        return pc
    
    def _get_kitti_poseset(self, idx):
        data = self.dataset[idx]
        sequence = '%02d'%data['seq']
        anc_idx = data['anc_idx']
        pos_idx = data['pos_idx']
        pc0 = self._get_kitti_data(sequence, anc_idx)
        pc1 = self._get_kitti_data(sequence, pos_idx)
        return pc0, pc1
    
    def __len__(self):
        return len(self.dataset)
    
    def set_fpfh_radiuse(self, radiuse, descriptors_radiuse):
        self.fpfh_normal_radiuse = radiuse
        self.fpfh_descriptors_radiuse = descriptors_radiuse
    
    def comute_FPFH(self, pc, kp):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc[:, :3])

        kp_num = kp.shape[0]
        pcd_kp = o3d.geometry.PointCloud()
        pcd_kp.points = o3d.utility.Vector3dVector(kp[:, :3])
        pcd += pcd_kp

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=30))

        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=100))
        fpfh_desc = np.transpose(pcd_fpfh.data)[-kp_num:]
        fpfh_desc = torch.tensor(fpfh_desc, dtype=torch.double)
        return fpfh_desc
    
    def test_FPFH(self, seq, idx):
        sequence = '%02d'%seq
        pc = self._get_kitti_data(sequence, idx)
        kp = self.kp[sequence][idx]
        fpfh_desc = self.comute_FPFH(pc, kp)
        return fpfh_desc
    
    def get_data(self, seq, idx):
        # data = self.dataset[idx]
        sequence = '%02d'%seq

        pose = torch.tensor(self.pose[sequence][idx], dtype=torch.double)
        kp = torch.tensor(self.kp[sequence][idx], dtype=torch.double)
        desc = torch.tensor(self.desc[sequence][idx], dtype=torch.double)
        score = torch.tensor(self.scores[sequence][idx], dtype=torch.double)
        pc = self._get_kitti_data(sequence, idx)
        fpfh_desc = self.comute_FPFH(pc, kp)

        T_cam0_velo = self.calib[sequence]
        T_cam0_velo = torch.tensor(T_cam0_velo, dtype=torch.double)

        '''transform point cloud from cam0 to LiDAR'''
        kpw = torch.einsum('ki,ij,jm->mk', pose, T_cam0_velo, kp.T)
        pcw = torch.einsum('ki,ij,jm->mk', pose, T_cam0_velo, pc.T)

        top_k = 10
        top_scores, top_indices = torch.topk(score, k=top_k)
        kp = kp[top_indices]
        kpw = kpw[top_indices]
        desc = desc[top_indices]

        norm = np.linalg.norm(desc, axis=1)
        norm = norm.reshape(len(kp), 1)
        desc = np.where(norm != 0, np.multiply(desc, 1/norm), 0)

        fpfh_norm = np.linalg.norm(fpfh_desc, axis=1)
        fpfh_norm = fpfh_norm.reshape(len(fpfh_desc), 1)
        fpfh_desc = np.where(fpfh_norm != 0, np.multiply(fpfh_desc, 1/fpfh_norm), 0)

        return {
            # 'skip': False,
            'keypoints': kp,
            'keypointsw': kpw,
            'descriptors': desc,
            'fpfh_descriptors': fpfh_desc,
            'scores': score,
            'sequence': sequence,
            'idx': idx,
            'pose': pose,
            'T_cam0_velo': T_cam0_velo,
            'cloud': pc,
            'cloudw': pcw,
        } 
    
    def get_pair_data(self, idx):
        data = self.dataset[idx]
        sequence = '%02d'%data['seq']
        anc_idx = data['anc_idx']
        pos_idx = data['pos_idx']
        pose0 = torch.tensor(self.pose[sequence][anc_idx], dtype=torch.double)
        pose1 = torch.tensor(self.pose[sequence][pos_idx], dtype=torch.double)

        kp0 = torch.tensor(self.kp[sequence][anc_idx], dtype=torch.double)
        kp1 = torch.tensor(self.kp[sequence][pos_idx], dtype=torch.double)

        desc0 = torch.tensor(self.desc[sequence][anc_idx], dtype=torch.double)
        desc1 = torch.tensor(self.desc[sequence][pos_idx], dtype=torch.double)

        score0 = torch.tensor(self.scores[sequence][anc_idx], dtype=torch.double)
        score1 = torch.tensor(self.scores[sequence][pos_idx], dtype=torch.double)

        pc0, pc1 = self._get_kitti_poseset(idx)
        pc0 = pc0.clone().detach()
        pc1 = pc1.clone().detach()

        fpfh_desc0 = self.comute_FPFH(pc0, kp0)
        fpfh_desc1 = self.comute_FPFH(pc1, kp1)

        T_cam0_velo = self.calib[sequence]
        T_cam0_velo = torch.tensor(T_cam0_velo, dtype=torch.double)
        T_gt = torch.einsum('ab,bc,cd,de->ae', torch.inverse(T_cam0_velo), torch.inverse(pose0), pose1, T_cam0_velo) # T_gt: transpose kp2 to kp1

        '''transform point cloud from cam0 to LiDAR'''
        kp0w_np = torch.einsum('ki,ij,jm->mk', pose0, T_cam0_velo, kp0.T)
        kp1w_np = torch.einsum('ki,ij,jm->mk', pose1, T_cam0_velo, kp1.T)
        pc0w = torch.einsum('ki,ij,jm->mk', pose0, T_cam0_velo, pc0.T)
        pc1w = torch.einsum('ki,ij,jm->mk', pose1, T_cam0_velo, pc1.T)

        fpfh_desc0 = torch.tensor(self.comute_FPFH(pc0, kp0), dtype=torch.double)
        fpfh_desc0 = torch.tensor(self.comute_FPFH(pc1, kp1), dtype=torch.double)

        '''transform pose from cam0 to LiDAR'''
        kp0w_np = kp0w_np[:, :3]
        kp1w_np = kp1w_np[:, :3]
        dists = cdist(kp0w_np, kp1w_np)
        '''Find ground true keypoint matching'''
        min0 = np.argmin(dists, axis=0)
        min1 = np.argmin(dists, axis=1)
        min0v = np.min(dists, axis=1)
        min0f = min1[min0v < self.threshold]
        '''For calculating repeatibility'''
        rep = len(min0f)
        '''
        If you got high-quality keypoints, you can set the 
        mutual_check to True, otherwise, it is better to 
        set to False
        '''
        match0, match1 = -1 * np.ones((len(kp0)), dtype=np.int16), -1 * np.ones((len(kp1)), dtype=np.int16)
        if self.mutual_check:
            xx = np.where(min1[min0] == np.arange(min0.shape[0]))[0]
            matches = np.intersect1d(min0f, xx)

            match0[min0[matches]] = matches
            match1[matches] = min0[matches]
        else:
            match0[min0v < self.threshold] = min0f

            min1v = np.min(dists, axis=0)
            min1f = min0[min1v < self.threshold]
            match1[min1v < self.threshold] = min1f

        kp0 = kp0[:, :3]
        kp1 = kp1[:, :3]

        norm0, norm1 = np.linalg.norm(desc0, axis=1), np.linalg.norm(desc1, axis=1)
        norm0, norm1 = norm0.reshape(len(kp0), 1), norm1.reshape(len(kp1), 1)
        desc0, desc1  = np.multiply(desc0, 1/norm0), np.multiply(desc1, 1/norm1)
        # desc0, desc1 = np.where(norm0 != 0, np.multiply(desc0, 1/norm0), 0), np.where(norm1 != 0, np.multiply(desc1, 1/norm1), 0)

        fpfh_norm0, fpfh_norm1 = np.linalg.norm(fpfh_desc0, axis=1), np.linalg.norm(fpfh_desc1, axis=1)
        fpfh_norm0, fpfh_norm1 = fpfh_norm0.reshape(len(kp0), 1), fpfh_norm1.reshape(len(kp1), 1)
        fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)
        # fpfh_desc0, fpfh_desc1  = np.multiply(fpfh_desc0, 1/fpfh_norm0), np.multiply(fpfh_desc1, 1/fpfh_norm1)

        fpfh_desc0 = torch.tensor(self.comute_FPFH(pc0, kp0), dtype=torch.double)
        fpfh_desc0 = torch.tensor(self.comute_FPFH(pc1, kp1), dtype=torch.double)
        
        return {
            # 'skip': False,
            'keypoints0': kp0,
            'keypoints1': kp1,
            'keypointsw0': kp0w_np,
            'keypointsw1': kp1w_np,
            'descriptors0': desc0,
            'descriptors1': desc1,
            'fpfh_descriptors0': fpfh_desc0,
            'fpfh_descriptors1': fpfh_desc1,
            'scores0': score0,
            'scores1': score1,
            'gt_matches0': match0,
            'gt_matches1': match1,
            'sequence': sequence,
            'idx0': anc_idx,
            'idx1': pos_idx,
            'pose0': pose0,
            'pose1': pose1,
            'T_cam0_velo': T_cam0_velo,
            'T_gt': T_gt,
            'cloud0': pc0,
            'cloud1': pc1,
            'cloudw0': pc0w,
            'cloudw1': pc1w,
            # 'all_matches': list(all_matches),
            # 'file_name': file_name
            'rep': rep
        } 


디스크립터가 0가 나오는지 확인해보자

In [65]:
test_fpfh_normal_radiuses = [1.0]
test_fpfh_descriptors_radiuses = [1.5]

In [67]:
# # test_fpfh_normal_radiuses = [0.1, 0.2, 0.4, 0.6, 0.8]
# # test_fpfh_descriptors_radiuses = [0.25, 0.5, 0.7, 1.0, 1.2, 1.5]
# test_fpfh_normal_radiuses = [0.05]
# test_fpfh_descriptors_radiuses = [0.15]
opt = argparse.Namespace(
    fpfh_normal_radiuse = 0.1,
    fpfh_descriptors_radiuse = 0.25,
    seq_list = [0],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    memory_is_enough = True,
    mutual_check = False,
    transform_opt = 0
)
data_set = mdgat_kitti_data_set(opt)

entire_norm_num_list = []
zeros_norm_num_list = []
max_zeros_num_list = []

for fpfh_normal_radiuse in test_fpfh_normal_radiuses:
    for fpfh_descriptors_radiuse in test_fpfh_descriptors_radiuses:
        data_set.set_fpfh_radiuse(fpfh_normal_radiuse, fpfh_descriptors_radiuse)
        for seq in opt.seq_list:
            seq_norm_num = 0
            seq_zeros_num = 0
            max_zeros_num = 0
            sequence = '%02d'%seq

            for idx in range(0, len(data_set.kp[sequence]), 100):
                data = data_set.get_data(seq, idx)
                desc = data['fpfh_descriptors']
                # print(desc)
                norm = np.linalg.norm(desc, axis=1)
                seq_norm_num += len(norm)
                seq_zeros_num += len(norm) - np.count_nonzero(norm)
                if max_zeros_num < len(norm) - np.count_nonzero(norm):
                    max_zeros_num = len(norm) - np.count_nonzero(norm)
            entire_norm_num_list.append(seq_norm_num)
            zeros_norm_num_list.append(seq_zeros_num)
            max_zeros_num_list.append([max_zeros_num, 256])

        for seq in opt.seq_list:
            # print((entire_norm_num_list[seq] / zeros_norm_num_list[seq]))
            print(fpfh_normal_radiuse, fpfh_descriptors_radiuse, "sequence %02d" % seq, 
                ", entire norm num: %d" % entire_norm_num_list[seq], 
                ", zeros norm num: %d" % zeros_norm_num_list[seq], 
                  ", ratio of zeros: %.4f" % (zeros_norm_num_list[seq] / entire_norm_num_list[seq]), 
                ", max zeros num: %d / %d" % (max_zeros_num_list[seq][0], max_zeros_num_list[seq][1]))


Data loaded
1.0 1.5 sequence 00 , entire norm num: 11776 , zeros norm num: 0 , ratio of zeros: 0.0000 , max zeros num: 0 / 256


모든 디스크립터가 0가 나오는 경우가 매우 줄었다. 그럼이제 다시 최적의 파라미터값을 찾는걸 해보자.

In [ ]:
opt = argparse.Namespace(
    fpfh_normal_radiuse = 0.5,
    fpfh_descriptors_radiuse = 1.25,
    seq_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    memory_is_enough = True,
    mutual_check = True,
    transform_opt = 0
)
data_set = mdgat_kitti_data_set(opt)
data = data_set.get_pair_data(0)

desc0 = data['descriptors0']
desc1 = data['descriptors1']
match0 = data['gt_matches0']
match1 = data['gt_matches1']

for idx, matched_idx in enumerate(match0):
    if matched_idx != -1:
        print('matched_idx: ', idx, ' & ', matched_idx, ' || distance: ', np.linalg.norm(desc0[idx] - desc1[matched_idx]))
        distance = np.linalg.norm(desc0 - desc1[matched_idx], axis=1)
        print('distance with matched points: ', distance[idx], ' || similarity rank: ', np.argsort(distance).tolist().index(idx), '/', len(distance))
        print('min distance: ', np.min(distance))
        print('mean distance: ', np.mean(distance))
        print('----------------------------------')

In [ ]:
from scipy.spatial import cKDTree

def extract_fpfh(pcd, voxel_size):
  radius_normal = voxel_size * 2
  pcd.estimate_normals(
      o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

  radius_feature = voxel_size * 5
  fpfh = o3d.pipelines.registration.compute_fpfh_feature(
      pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
  return np.array(fpfh.data).T

def pcd2xyz(pcd):
    return np.asarray(pcd.points).T

def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
    feat1tree = cKDTree(feat1)
    dists, nn_inds = feat1tree.query(feat0, k=knn)
    if return_distance:
        return nn_inds, dists
    else:
        return nn_inds
    
def find_correspondences1(feats0, feats1):
    dists = cdist(feats0, feats1)

    '''Find ground true keypoint matching'''
    min1 = np.argmin(dists, axis=0)
    min2 = np.argmin(dists, axis=1)
    min1v = np.min(dists, axis=1)
    min1f = min2[min1v < 0.5]

    match1, match2 = -2 * np.ones((len(feats0)), dtype=np.int16), -2 * np.ones((len(feats1)), dtype=np.int16)
    xx = np.where(min2[min1] == np.arange(min1.shape[0]))[0]
    matches = np.intersect1d(min1f, xx)

    match1[min1[matches]] = matches
    match2[matches] = min1[matches]

    return match1, match2

def find_correspondences(feats0, feats1, mutual_filter=True):
    """
    Find correspondences between two sets of features.

    Args:
        feats0 (numpy.ndarray): Array of features from the first set.
        feats1 (numpy.ndarray): Array of features from the second set.
        mutual_filter (bool, optional): Whether to apply mutual filtering. Defaults to True.

    Returns:
        tuple: A tuple containing two numpy arrays: corres_idx0 and corres_idx1.
               corres_idx0 (numpy.ndarray): Array of indices of correspondences in feats0.
               corres_idx1 (numpy.ndarray): Array of indices of correspondences in feats1.
    """
    # print(feats0.shape, feats1.shape)
    nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
    # print(nns01.shape)
    corres01_idx0 = np.arange(len(nns01))
    corres01_idx1 = nns01

    if not mutual_filter:
        return corres01_idx0, corres01_idx1

    nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
    corres10_idx1 = np.arange(len(nns10))
    corres10_idx0 = nns10

    # print(corres01_idx0, corres01_idx1)

    match1, match2 = -1 * np.ones((len(feats0)), dtype=np.int16), -1 * np.ones((len(feats1)), dtype=np.int16)

    mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
    # print(mutual_filter)
    corres01_idx0[~mutual_filter] = -2
    corres01_idx1[~mutual_filter] = -2
    # print(corres01_idx0)
    
    # corres_idx0 = corres01_idx0[mutual_filter]
    # print(corres_idx0)
    # corres_idx1 = corres01_idx1[mutual_filter]

    return corres01_idx0, corres01_idx1

In [90]:
# test_fpfh_normal_radiuses = [0.4, 0.5, 0.6, 0.7]
# test_fpfh_descriptors_radiuses = [0.7 ,1.0, 1.2, 1.5]
test_fpfh_normal_radiuses = [1.0]
test_fpfh_descriptors_radiuses = [1.5]
# test_fpfh_descriptors_radiuses = [1.5]

opt = argparse.Namespace(
    fpfh_normal_radiuse = 1.0,
    fpfh_descriptors_radiuse = 1.5,
    seq_list = [0, 2],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    memory_is_enough = True,
    mutual_check = False,
    transform_opt = 0
)
data_set = mdgat_kitti_data_set(opt=opt)
correct = 0
total = 0

for i in range(0, len(data_set), 100000):
    data = data_set.get_pair_data(i)
    kp0 = data['keypoints0']
    kp1 = data['keypoints1']
    desc0 = data['descriptors0']
    desc1 = data['descriptors1']
    fpfh_desc0 = data['fpfh_descriptors0']
    fpfh_desc1 = data['fpfh_descriptors1']
    match0_gt = data['gt_matches0']
    match1_gt = data['gt_matches1']

    # np.set_printoptions(threshold=np.inf)

    print(fpfh_desc0.shape, fpfh_desc1.shape)

    distances = torch.cdist(fpfh_desc0, fpfh_desc1, p=2)
    print(distances)
    match0 = torch.argmin(distances, dim=1).numpy()
    match1 = torch.argmin(distances, dim=0).numpy()
    print("match0:", match0.shape, match0_gt.shape)
    print("match1:", match)

    corres0 = match0 == match0_gt
    corres1 = match1 == match1_gt

    print(fpfh_desc0)

    print("corres0:", corres0)
    print("corres1:", corres1)

Data loaded


/tmp/ipykernel_112045/262698142.py:226: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  fpfh_desc0 = torch.tensor(self.comute_FPFH(pc0, kp0), dtype=torch.double)
/tmp/ipykernel_112045/262698142.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  fpfh_desc0 = torch.tensor(self.comute_FPFH(pc1, kp1), dtype=torch.double)
/tmp/ipykernel_112045/262698142.py:272: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  fpfh_desc0 = torch.tensor(self.comute_FPFH(pc0, kp0), dtype=torch.double)


torch.Size([256, 33]) (256, 33)


/tmp/ipykernel_112045/262698142.py:273: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  fpfh_desc0 = torch.tensor(self.comute_FPFH(pc1, kp1), dtype=torch.double)


TypeError: cdist(): argument 'x2' (position 2) must be Tensor, not numpy.ndarray